![DeepNeuro](https://github.com/QTIM-Lab/DeepNeuro/raw/master/package_resources/logos/DeepNeuro_alt.PNG?raw=true)

# Running a DeepNeuro Module with Docker

In this notebook, you will learn how to run a DeepNeuro module from its Docker container. DeepNeuro modules are scripts created with DeepNeuro syntax, and their goal is usually to: load data from any state of preprocessing, run inference on a model trained to accomplish a certain task, and save out to the results to a location of your choosing. Today, we will be running the glioma segmentation module in DeepNeuro.

You will need to have Docker already installed for this tutorial. You can find some instructions on how to do that here: https://docs.docker.com/install/. Some tutorials on how to use Docker can be found here: https://docker-curriculum.com/.

In order to run these Docker containers on the GPU, you will also have to install nvidia-docker. nvidia-docker is an extension to Docker that lets you seamlessly hook up your docker containers to your NVIDIA GPU drivers and supporting software. You can find instructions on how to install nvidia-docker here: https://github.com/NVIDIA/nvidia-docker.

You can find documentation for the glioma segmentation module at this link: https://github.com/QTIM-Lab/DeepNeuro/tree/master/deepneuro/pipelines/Segment_GBM. Don't worry about reading it too much right now -- we will go over the parameters in this tutorial too.

Our first step will be to pull the Docker container for glioma segmentation with DeepNeuro.

In [3]:
!docker pull qtimlab/deepneuro_segment_gbm

Using default tag: latest
latest: Pulling from qtimlab/deepneuro_segment_gbm
Digest: sha256:890f03b339354f3a35aadf98045773f23d13b15ddf11df5680c7d3d68247cb3f
Status: Image is up to date for qtimlab/deepneuro_segment_gbm:latest


Easy enough! We will also have to download our medical imaging data. If you haven't already downloaded it in a previous tutorial, you can get download it from DeepNeuro with the following command. We will be loading a DICOM dataset this time, instead of a NIfTI dataset, to show off some of the preprocessing steps that come prepackaged in DeepNeuro Docker containers.

In [4]:
from deepneuro.load.load import load

load('sample_gbm_dicom', output_datapath='./Sample_Data')

KeyError: 'sample_gbm_dicom'

In the folder you just downloaded, you should see four subfolders corresponding to T2, FLAIR, pre-contrast T1, and post-contrast T1 MR sequences. These images are not the same resolution, are not registered to the same patient space, are not on the same intensity scale, and have not otherwise been preprocessed. Luckily, we have a Docker container that can do all of those things, and even segment glioma tissue at the end of the process!

There are two principle ways to run a Docker container in DeepNeuro. One is directly via the Docker interface: simply construct a command and enter it into the command-line. The other is via DeepNeuro's Docker wrapper, written in Python. The latter takes care of some of the difficulties of working with Docker, and let's you run Docker containers from within a Python script, but requires you to have a local installation of DeepNeuro.

## Running a module via Docker and the Command Line